# Build Recsys based on popularity model

In [21]:
import json
from pandas.io.json import json_normalize

import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

## Loading interaction data

In [22]:
def filter_by_hours_played(path,minutes_played=None,percentile=None):
    with open(path) as f:
        df_items = json.load(f)    
    parsed_items = json_normalize(data=df_items, 
                                  record_path='items', 
                                  meta=['items_count','steam_id','user_id','user_url'])
    if minutes_played != None:
        parsed_items = parsed_items[parsed_items.playtime_forever > minutes_played]
    if percentile != None:
        quantiles = parsed_items.groupby('item_name')['playtime_forever'].quantile(q=percentile)
        parsed_items = pd.merge(parsed_items, pd.DataFrame(quantiles), how='left', left_on='item_name', right_index=True)
        parsed_items = parsed_items.loc[parsed_items['playtime_forever_x'] > parsed_items['playtime_forever_y']]
    return parsed_items

In [24]:
## reading australian_users_items_fixed.json 

folder = "./"
interactions_df = filter_by_hours_played(folder + 'data/australian_users_items_fixed.json', 0)


<ipython-input-22-9d53480a1560>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  parsed_items = json_normalize(data=df_items,


In [25]:
# drop duplicates
interactions_full_df.drop_duplicates()

,item_id,user_id,count
0,10,76561197970982479,1
2,30,76561197970982479,1
8,300,76561197970982479,1
9,240,76561197970982479,1
10,3830,76561197970982479,1
...,...,...,...
5153202,304930,76561198329548331,1
5153203,227940,76561198329548331,1
5153206,388490,76561198329548331,1
5153207,521570,76561198329548331,1


In [26]:
## drop the other column
interactions_full_df= interactions_df.drop(["item_name","steam_id", "playtime_forever","playtime_2weeks", "items_count","user_url"], axis = 1)

In [27]:
interactions_full_df.describe()

,item_id,user_id
count,3285246,3285246
unique,10050,68403
top,730,chidvd
freq,43279,3475


In [28]:
interactions_full_df["count"] = 1

# Build popularity dataframe

In [29]:
interactions_full_df.head()

,item_id,user_id,count
0,10,76561197970982479,1
2,30,76561197970982479,1
8,300,76561197970982479,1
9,240,76561197970982479,1
10,3830,76561197970982479,1


In [30]:
### swap  column 1 and 2
first_col = interactions_full_df.pop("user_id")
interactions_full_df.insert(0,"user_id", first_col)

In [31]:
interactions_full_df.head()

,user_id,item_id,count
0,76561197970982479,10,1
2,76561197970982479,30,1
8,76561197970982479,300,1
9,76561197970982479,240,1
10,76561197970982479,3830,1


In [32]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('item_id')['count'].count().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,item_id,count
0,730,43279
1,4000,42074
2,550,34899
3,304930,34465
4,105600,28541
5,620,22696
6,218620,22378
7,240,21671
8,72850,21642
9,230410,21087


In [33]:
## change column label
item_popularity_df = item_popularity_df.rename(columns={"count":"popularity"})

In [34]:
item_popularity_df.to_csv('./popularity.csv', index=False)  

## recommend by poplarity by cold start uer

In [40]:
def rec_popularity(k = 10):
    return item_popularity_df["item_id"].head(k)

In [41]:
rec_popularity()

0       730
1      4000
2       550
3    304930
4    105600
5       620
6    218620
7       240
8     72850
9    230410
Name: item_id, dtype: object